In [0]:
import cvxpy as cp
import numpy as np
#pip install CVXPY

1 Using CVXPY

In [0]:
# Create two scalar optimization variables.
x = cp.Variable()
y = cp.Variable()

In [0]:
# Create two constraints.
constraints = [x + y == 1,
x - y >= 1]

In [0]:
# Form objective.
obj = cp.Minimize((x - y)**2)

In [0]:
# Form and solve problem.
prob = cp.Problem(obj, constraints)

In [0]:
prob.solve() # Returns the optimal value.

1.0

In [0]:
print("status:", prob.status)

status: optimal


In [0]:
print("optimal value", prob.value)

optimal value 1.0


In [0]:
print("optimal var", x.value, y.value)

optimal var 1.0 1.570086213240983e-22


# 2. UNCONSTRAINED OPTIMIZATION

In [0]:
# 2.1 Problem 1
x1 = cp.Variable(1)
x2 = cp.Variable(1)

#vector variables
#x1 = cp.Variable(1)
#x2 = cp.Variable(1)
#using one dimensional vector variables, the result is identical as when used scalar variables
#so all computations are done using scalar variables
#there is an error when higher dimensional variables are used

In [0]:
obj1 = cp.Minimize((x1-4)**2 + 7*(x2-4)**2 + 4*x2) #objective function

In [0]:
prob1 = cp.Problem(obj1)

In [0]:
prob1.solve()

15.428571428571429

In [0]:
print("status:", prob1.status)
print("optimal value", prob1.value)
print("optimal var x1 = ", x1.value)
print("optimal var x2 = ", x2.value)
#solution with scalar variables
#status: optimal
#optimal value 15.428571428571429
#optimal var x1 =  4.0
#optimal var x2 =  3.7142857142857144 #the calculated variables are scalars

#the problem when solved on paper, gives the exact same result as above.

#solution when used vector variables
#status: optimal
#optimal value 15.428571428571429
#optimal var x1 =  [4.]
#optimal var x2 =  [3.71428571] #the variables are vectors in this case
#since the results are identical for all the problems, calulation for vector variables
#is not considered.

status: optimal
optimal value 15.428571428571429
optimal var x1 =  [4.]
optimal var x2 =  [3.71428571]


2.2 Problem 2

In [0]:
from sympy import *
import numpy as np

In [0]:
#check convexity
x1 = Symbol('x1')
x2 = Symbol('x2')
x3 = Symbol('x3')
# initialising variables using SYMPY module

In [0]:
f1 = x1**3 + (x2-x3)**2 + x3**3 + 2 # function

In [0]:
dx1 = f1.diff(x1) #6*x1**2
dx2 = f1.diff(x2) #36*x2 - 12*x3
dx3 = f1.diff(x3) #-12*x2 + 4*x3 - 4
# finding jacobian of f1 using sympy diff method

In [0]:
dx1

3*x1**2

In [0]:
dx2

2*x2 - 2*x3

In [0]:
dx3

-2*x2 + 3*x3**2 + 2*x3

In [0]:
hessian = Matrix(np.array([[dx1.diff(x1),dx1.diff(x2),dx1.diff(x3)],
                           [dx2.diff(x1),dx2.diff(x2),dx2.diff(x3)],
                           [dx3.diff(x1),dx3.diff(x2),dx3.diff(x3)]]))
# calculating hessian matrix using sympy methods

In [0]:
hessian.det() #calculating det of hessian

12*x1*(6*x3 + 2) - 24*x1

In [0]:
hessian.trace() #calculating trace of hessian
#constraints for the problem to be convex can be found by computing determinant >= 0 and trace >= 0
# the two constraints thus found are -
# x1*x3 >= 0
# 3*x1 + 3*x3 + 2 >= 0
# The problem is convex if it satisfies the above mentioned constraints.

6*x1 + 6*x3 + 4

In [0]:
#formulate LPP
x1 = cp.Variable()
x2 = cp.Variable()
x3 = cp.Variable()

In [0]:
obj2 = cp.Minimize(x1**3 + (x2-x3)**2 + x3**3 + 2)

In [0]:
prob2 = cp.Problem(obj2)
prob2.solve()

1.9999999993629338

In [0]:
print("status:", prob2.status)
print("optimal value", prob2.value)
print("optimal var", x1.value, x2.value,x3.value)

#status: optimal
#optimal value 1.9999999993629338
#optimal var 0.0008258815459611968 0.0008258861793499652 0.0008258861755015717
# solving the problem traditional way gives the solution as
# x1 = x2 = x3 = 0 and the optimal value = 2 which is close to the solution found by the CVXPY

status: optimal
optimal value 1.9999999993629338
optimal var 0.0008258815459611968 0.0008258861793499652 0.0008258861755015717


2.3 Problem 3

In [0]:
x1 = cp.Variable()
x2 = cp.Variable()

In [0]:
obj3 = cp.Minimize((x1-2)**2 + 3*x2)
const3 = [x1+x2-4<=0]
prob3 = cp.Problem(obj3,const3)
prob3.solve()
#solution to the problem without log barrier function shows unbounded status and the function value goes to negative infinity
#status: unbounded
#optimal value -inf
#optimal var None None

-inf

In [0]:
#reformulating using the log barrier function
objlog = cp.Minimize((x1-2)**2 + 3*x2 - 0.001*cp.log(-(-x1-x2+4))) #log barrier is added to the objective function
#parameter is chosen as 0.001, 1, 100 and 100000
# NOTE -  the parameter is multiplied directly to the barrier function in the objective function instead of using 1/t

In [0]:
problog = cp.Problem(objlog)
problog.solve()

#solution when parameter = 0.001
#status: optimal
#optimal value 3.759006322763877
#optimal var 3.500000963375165 0.500332323592533

#solution when parameter = 1
#status: optimal
#optimal value 5.84861227238681
#optimal var 3.500000398379134 0.8333329278362519  

#solution when parameter = 100
#status: optimal
#optimal value -246.9057897847487
#optimal var 3.499976905547963 33.83335747486957

#solution when parameter = 100000
#status: optimal_inaccurate
#optimal value -941427.573319449
#optimal var 3.4999982666221587 33333.833961723896

# as the size of the parameter is increased, the optimal value of the solution..
# ..moves towards negative infinity. with this we can bound the function as per requirement.
# in other words, as we decrease the parameter size, we get closer to the optimal solution
# here the optimal value of parameter is close to 0.001 which gives the best solution for this problem

3.759006322763877

In [0]:
print("status:", problog.status)
print("optimal value", problog.value)
print("optimal var", x1.value, x2.value)

status: optimal
optimal value 3.759006322763877
optimal var 3.500000963375165 0.500332323592533


3 Modeling constrained problems

3.1 Water resources

In [0]:
r = cp.Variable() # r is no. of liters drawn from reservoir
s = cp.Variable() # s is no. of liters drawn from stream

In [0]:
obj_cost = cp.Minimize((r/10)+(5*s/100)) #minimizing total cost

In [0]:
const_water = [r+s==500000,
               s<=100000,
               50*r + 250*s <= 50000000] #constraints
# ppm is mg/liter so the third constraint is formulated in such as way that the units are same for all variables
# so 100 ppm is multiplied with 500000 liters of water

In [0]:
prob_water = cp.Problem(obj_cost,const_water)
prob_water.solve()

45000.000000000015

In [0]:
print("status:", prob_water.status)
print("optimal value", prob_water.value)
print("optimal var r =", r.value)
print("optimal var s =", s.value)

#status: optimal
#optimal value 45000.000000000015
#optimal var r = 400000.0000000001
#optimal var s = 100000.00000000003

status: optimal
optimal value 45000.000000000015
optimal var r = 400000.0000000001
optimal var s = 100000.00000000003


3.2 Good-smelling perfume design

In [0]:
b1 = cp.Variable() # fraction of blend 1 in perfume
b2 = cp.Variable() # fraction of blend 2
b3 = cp.Variable() # fraction of blend 3
b4 = cp.Variable() # fraction of blend 4

In [0]:
obj_perfcost = cp.Minimize(55*b1 + 65*b2 + 35*b3 +85*b4) # minimizing the total cost

In [0]:
total = b1+b2+b3+b4 # total fraction adds up to 1 (used in constraints)

In [0]:
a1 = b2
a2 = b3
a3 = b1
a4 = 0.35*b1+0.60*b2+0.35*b3+0.40*b4
a5 = 0.15*b1+0.05*b2+0.20*b3+0.10*b4
a6 = 0.30*b1+0.20*b2+0.40*b3+0.20*b4
a7 = 0.20*b1+0.15*b2+0.05*b3+0.30*b4 #used in constraint matrix

In [0]:
const_perf = [
              0.05<=a1, a1<=0.2,
              0.3<=a2,
              0.1<=a3, a3<=0.25,
              a4 <=0.5,
              0.08<=a5, a5 <=0.13,
              a6 <=0.35,
              0.19<= a7,total ==1
]
#all the constraints are formulated as mentioned in the problem

In [0]:
prob_perf = cp.Problem(obj_perfcost,const_perf)

In [0]:
prob_perf.solve()

63.0

In [0]:
print("status:", prob_perf.status)
print("optimal value", prob_perf.value)
print("optimal var b1 =", b1.value)
print("optimal var b2 =", b2.value)
print("optimal var b3 =", b3.value)
print("optimal var b4 =", b4.value)

#status: optimal
#optimal value 63.0
#optimal var b1 = 0.14
#optimal var b2 = 0.139
#optimal var b3 = 0.3
#optimal var b4 = 0.42
# as the optimal solution is having all positive fractions, so the solution is valid
# the optimal cost is 63 euros.

status: optimal
optimal value 63.0
optimal var b1 = 0.14000000000000032
optimal var b2 = 0.13999999999999976
optimal var b3 = 0.3
optimal var b4 = 0.42


3.3 Roadway expenses

In [0]:
xr = cp.Variable() #money spend on rural projects
xu = cp.Variable() # money spent on urban projects

In [0]:
obj_road = cp.Maximize(7000*cp.log(1+xr) + 5000*cp.log(1+xu) - xr - xu) #objective function

In [0]:
const_road = [xu+xr ==200] #constraint

In [0]:
prob_road = cp.Problem(obj_road,const_road)

In [0]:
prob_road.solve()

55348.89310673721

In [0]:
br = 7000*np.log(1+xr.value) #benefit from rural spending
bu = 5000*np.log(1+xu.value) #benefit from urban spending

In [0]:
print("status:", prob_road.status)
print("optimal benefit =", prob_road.value)
print("optimal var xu =", xu.value)
print("optimal var xr =", xr.value)
print("optimal var bu =", bu)
print("optimal var br =", br)

#status: optimal
#optimal benefit = 55348.89310673721
#optimal var xu = 83.1666963297482
#optimal var xr = 116.83330317505175
#optimal var bu = 22163.996562399472
#optimal var br = 33384.896585098315 All numbers are in Millions
# optimal solution makes sense as all the values are positive
# optimal benefit is close to 55348 Million euros

status: optimal
optimal benefit = 55348.89310673721
optimal var xu = 83.1666963297482
optimal var xr = 116.83330317505175
optimal var bu = 22163.996562399472
optimal var br = 33384.896585098315


3.4 Design you own optimization problem

The problem is to maximize the "Taste Points" of a pizza. The taste points is like rating a pizza. The pizza receives 4 points per kg of olives used and 12 points per kg of cheese used. 
The max time to make a pizza is 81 minutes. Olives take 18 minutes to cook and cheese takes 9 minutes.
Total budget for making pizza is limited to 120 euros where olives cost 4 eur/kg and cheese costs 20 eur/kg.
What are the maximum amount of taste points that can be obtained for the pizza? And what is the quantity of olives and cheese used?

In [0]:
olives = cp.Variable()
cheese = cp.Variable()

In [0]:
obj_pizza = cp.Maximize(4*olives+12*cheese) #objective function is written to maximize the total taste points

In [0]:
const_pizza = [4*olives+20*cheese<=120,
               18*olives+9*cheese<=81]
#constraints are chosen from the time and cost factors mentioned in the question

In [0]:
prob_pizza = cp.Problem(obj_pizza,const_pizza) #the problem is built using objective function and constraints

In [0]:
prob_pizza.solve()

74.66666666666667

In [0]:
print("status:", prob_pizza.status)
print("Maximum taste points =", prob_pizza.value)
print("optimal olives =", x.value)
print("optimal cheese =", y.value)

#status: optimal
#Maximum taste points = 74.66666666666667
#optimal olives = 1.6666666666666663
#optimal cheese = 5.666666666666666 
#Units are in kg
# so the maximum taste points that can be achieved using the ingredients in 74.6

status: optimal
Maximum taste points = 74.66666666666667
optimal olives = 1.6666666666666663
optimal cheese = 5.666666666666666


4 Data Analysis

A simple linear regression problem is cconsidered where we need to predict the grade of students based on the number of hours they studied.
X(no of hours studied is ) = [5,3,0] and y(grade) = [9,6,1].
Data is provided for 3 students.

In [0]:
#4.1
A = np.array([[1,5],[1,3],[1,0]]) #A is the matrix with bias(chosen as 1) and x values
y = np.transpose(np.array([9,6,1])) #y is the labels

In [0]:
#CVXPY is used to minimise the cost function
teta = cp.Variable(2) #teta are the tuning parameters of the model
objfunction = cp.sum_squares(A@teta - y) #objective function is the cost function of the problem
prob = cp.Problem(cp.Minimize(objfunction)) #we minimise the cost
prob.solve()

0.02631578947368456

In [0]:
print("The optimal teta values =",teta.value) #the equation to predict the new grades can be found by using the equation, y_hat = optimal_teta*x_test + bias

# The optimal teta values = [1.05263158 1.60526316] first element is bias and second is teta which is coefficient of x variable

The optimal teta values = [1.1        1.59411765]


In [0]:
#4.2 
#CASE 1 - ADDING CONSTRAINTS ON TETA TO FIND ITS RANGE 
teta = cp.Variable(2) 
objfunction = cp.sum_squares(A@teta - y) 
constraints = [1.1 <= teta, teta <= 1.6] #constraints are added on teta values to find the range of teta which gives minimum cost
#constraints are chosen at random and then changed according to the result obtained (heuristic approach)
newobj = cp.Minimize(objfunction)
prob = cp.Problem(newobj, constraints)
prob.solve()
#The optimal teta values = [1.1 1.59411765] for the range of 1.1 <= teta <= 1.6

0.028823529411765088

In [0]:
# CASE 2 - USING POLYNOMIAL REGRESSION TO SOLVE THE SAME PROBLEM
#extra coefficients are added for x**2 in A matrix (same as adding new monomials in the cost function)
A = np.array([[1,5,25],[1,3,9],[1,0,0]])
y = np.transpose(np.array([9,6,1]))
teta = cp.Variable(3) 
objfunction = cp.sum_squares(A@teta - y) 
newobj = cp.Minimize(objfunction)
prob = cp.Problem(newobj)
prob.solve() # the minimum solution is almost zero in this case

2.706312172759759e-43

In [0]:
print("The optimal teta values(polynomial regression) =",teta.value)
#The optimal teta values(polynomial regression) = [ 1  1.76666667 -0.03333333]

The optimal teta values(polynomial regression) = [ 1.          1.76666667 -0.03333333]


In [0]:
# CASE 3 - USING LASSO OPTIMIZATION ON THE POLYNOMIAL REGRESSOR
A = np.array([[1,5,25],[1,3,9],[1,0,0]])
y = np.transpose(np.array([9,6,1]))
teta = cp.Variable(3) 
objfunction = cp.sum_squares(A@teta - y) + 0.01*cp.norm2(teta) 
#objective function is penalized with L2 norm of the teta. The parameter is chosen as 0.01
newobj = cp.Minimize(objfunction )
prob = cp.Problem(newobj)
prob.solve() 

0.020288275147731922

In [0]:
print("The optimal teta values(polynomial regression) =",teta.value)
#The optimal teta values(Lasso regression) = [ 0.99985714  1.76331598 -0.03260496]

The optimal teta values(polynomial regression) = [ 0.99985714  1.76331598 -0.03260496]
